In [1]:
import numpy as np
import scipy.stats as ss
import pandas as pd
import matplotlib
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')
%matplotlib inline

In [2]:
df = pd.read_csv('breakfasts.csv', na_values='*')
df.head(5)

,Unnamed: 0,Weight
0,0,20.440
1,1,20.244
2,2,20.549
3,3,20.755
4,4,20.740


In [3]:
df.describe()

,Unnamed: 0,Weight
count,10000.00000,10000.000000
mean,4999.50000,20.499212
std,2886.89568,0.199874
min,0.00000,19.752000
25%,2499.75000,20.365000
50%,4999.50000,20.500000
75%,7499.25000,20.635000
max,9999.00000,21.171000


In [4]:
df.shape

(10000, 2)

In [5]:
df[df['Weight'] < 20].shape

(55, 2)

In [6]:
from scipy.stats import norm

m = df['Weight'].mean()
st = df['Weight'].std()
1000 * norm.cdf(20, m, st)


6.251299672642689

In [7]:
m = df['Weight'].mean()
st = df['Weight'].std()

win = 42000 * 35274 * (0.75 / m)
exp = 1000 * norm.cdf(20, m, st)
loss = 250000 * exp
win - loss


52640769.488209896

In [8]:
def try_m(me):
    win = 42000 * 35274 * (0.75 / me)
    exp = 1000 * norm.cdf(20, me, st)
    loss = 250000 * exp
    return 2 * (loss - win)

def try_m_st_half(me):
    win = 42000 * 35274 * (0.75 / me)
    exp = 1000 * norm.cdf(20, me, st / 2)
    loss = 250000 * exp
    return 2 * (loss - win)

In [9]:
from scipy.optimize import minimize
res_full = minimize(try_m, x0=m, bounds=[[19, 30]])
print(res_full)
res = minimize(try_m_st_half, x0=m, bounds=[[19, 30]])
print(res)

      fun: array([-1.07329321e+08])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([220.53716742])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 20
      nit: 4
     njev: 10
   status: 0
  success: True
        x: array([20.64795862])
      fun: array([-1.09090003e+08])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([17.88139195])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 32
      nit: 7
     njev: 16
   status: 0
  success: True
        x: array([20.34382338])


In [10]:
print(-res.fun[0] + res_full.fun[0] - 1e6)

760682.0997877419


# Последнее задание.


In [11]:
n = 10000
zq = norm.ppf(0.001)
f = []
s = []
for i in tqdm(range(50000)):
    nor = ss.norm.rvs(size=n) # random variates, location, scale
    data = pd.DataFrame({'norm': nor})
    m, st = data.agg([np.mean, np.std])['norm']
    f.append(norm.cdf(zq, m, st))
    s.append(nor[nor < zq].size / n)

f = np.array(f)
s = np.array(s)
print(s.std() / f.std())





3.8722209851125564
